**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 4 - Open Science Platform - Cross link User and Project accounts

1 - Load required modules

In [21]:
from new_helper import *
from super_helper import *
from loguru import logger

2 - Set variables and values

In [22]:
# Index for objects in both user account and project account JSON-LDs.
json_ld_index = 1

# Local path for file upload
directory_path = "upload"

# Directory for file downloads
download_path = "download"

# Read accounts from JSON-LD
user_accounts, user_account_short_ids  = read_user_accounts_from_jsonld('logs/accounts.json')
project_accounts, project_account_short_ids = read_project_accounts_from_jsonld('logs/projects.json')

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [23]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-10 15:06:25,547 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-10 15:06:25.549 | INFO     | __main__:<module>:3 - b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'


5 - Data extraction from JSON-LD.

Extracts account ids from `datasets/accounts.json` and `datasets/projects.json`.

Must update `json_ld_index` (see item 2) with a entry number related to the position of an existing object in `datasets/accounts.json` and `datasets/projects.json`. Example: `json_ld_index = 2`, selects the second object in each json file.

6 - Sets details for both User and Project accounts providing a logical link between them for later references.

In [24]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)



# Assuming json_ld_index is defined
user_account = user_accounts[json_ld_index]
user_account_short_id = user_account_short_ids[json_ld_index]
project_account = project_accounts[json_ld_index]
project_account_short_id = project_account_short_ids[json_ld_index]

logger.debug(f"{user_account['account_id']}")
logger.debug(f"{user_account_short_id['account_id']}")

# Set project_id as a detail for the user account
hash_user_to_project = set_account_detail(
    address, 
    user_account['account_id'], 
    "linked_project", 
    project_account['account_id']
)

# Set user_account_id as a detail for the project account
hash_project_to_user = set_account_detail(
    address, 
    project_account['account_id'], 
    "linked_user", 
    user_account['account_id']
)

# Update the JSON-LD files with the linked details
update_user_account_link(user_account['account_id'], project_account['account_id'])
update_project_account_link(project_account['account_id'], user_account['account_id'])

# Confirming the operation
logger.info(f"User account {user_account['account_id']} linked to project {project_account['account_id']}")
logger.info(f"Project account {project_account['account_id']} linked to user {user_account['account_id']}")

2025-02-10 15:06:25.610 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-10 15:06:25.629 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'
2025-02-10 15:06:25.630 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: A518E3EE4D45C20EF834361093305D2500A46228
2025-02-10 15:06:25.631 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-10 15:06:25.633 | DEBUG    | __main__:<module>:11 - optimistic_aryabhata@test
2025-02-10 15:06:25.634 | DEBUG    | __main__:<module>:12 - optimistic_aryabhata
2025-02-10 15:06:25.636 | DEBUG    | integration_helpers:tracer:35 - 	Entering "set_account_detail"
2025-02-10 15:06:25,652 - INFO - None
2025-02-10 15:06:25,655 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)


2025-02-10 15:06:28,131 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:06:28,139 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:06:28.141 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-10 15:06:28.161 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001
2025-02-10 15:06:28.162 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                                                                                
2025-02-10 15:06:28.164 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_result"
2025-02-10 15:06:28.166 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "set_account_detail"
2025-02-10 15:06:28.167 | DEBUG    | integration_helpers

7 - Load the sart contract hash

In [25]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)

2025-02-10 15:06:31,221 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-10 15:06:31.223 | INFO     | __main__:<module>:3 - b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-10 15:06:31.224 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-10 15:06:31.243 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-10 15:06:31.244 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 15:06:31.245 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"


8 - Queries Iroha 1 for the User account and exposes the variable `project_id` from user details.

In [26]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id['account_id'], "test")
address = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
# account_metadata_cid = process_raw_data(address) 

# Process the raw data
account_metadata_cid, project_id = process_raw_data(address) #TO DO - From now

2025-02-10 15:06:31.300 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-10 15:06:31,303 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 15:06:31,320 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-10 15:06:31,322 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:06:34,196 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:06:34,205 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:06:34,206 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '9b19c316b5d001197d5b5a4d8f435bcf0d8ede6fc8e816d7cae5c9f305bf5bf2', 'timestamp': 1739199994206}
2025-02-10 15:06:34.208 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-10 15:06:34,211 - INFO - Appended data under 'optimistic_aryabhata@test' in logs/account_data.json
2025-02-10 15:06:34.212 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-

9 - Queries Iroha 1 for the project account.

In [27]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, project_account_short_id['account_id'], "test")
details = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-02-10 15:06:34.299 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-10 15:06:34,301 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-10 15:06:34,319 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-10 15:06:34,321 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-10 15:06:37,236 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-10 15:06:37,246 - INFO - ('COMMITTED', 5, 0)
2025-02-10 15:06:37,248 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '005f4e9fccb4910f20bf36517f479896125c5c31ddfb496230c1e23f48386a37', 'timestamp': 1739199997248}
2025-02-10 15:06:37.249 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-10 15:06:37,252 - INFO - Appended data under '98639@test' in logs/account_data.json
2025-02-10 15:06:37.253 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-10 15:06:37.254

10 - Saves `project_id` in the local file sytem for use in Notebook 4

In [28]:
# Dumping the variable to JSON
dump_variable(project_id, "Project Id")

2025-02-10 15:06:37.299 | DEBUG    | integration_helpers:tracer:35 - 	Entering "dump_variable"
2025-02-10 15:06:37,303 - INFO - Variable 'Project Id' successfully dumped to temp/Project Id.json
2025-02-10 15:06:37.304 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "dump_variable"
